### Ingestion del archivo "person.json"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

#### Paso 1 - Leer el archivo JSON utilizando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
name_schema = StructType([
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

In [0]:
person_schema = StructType([
    StructField("personId", IntegerType(),False),
    StructField("personName", name_schema)
])

In [0]:
persons_df = spark.read \
    .schema(person_schema)\
    .json(f"{bronze_folder_path}/{v_file_date}/person.json")

#### Paso 2 - Cambiar el nombre de las columnas segun lo "requerido"

In [0]:
persons_renamed_df = persons_df \
    .withColumnRenamed("personId","person_id") \
    

#### Paso 3 - Agregar una columna "ingestion date" al dataframe

In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import lit
from pyspark.sql.functions import concat

In [0]:
person_final_df = add_ingestion_date(persons_renamed_df) \
.withColumn("enviroment", lit(dbutils.widgets.get("p_enviroment"))) \
.withColumn("file_date", lit(v_file_date))\
.withColumn("name",
            concat(
                "personName.forename",
                lit(" "),
                "personName.surname"
                )
            )

In [0]:
person_final_df_dropped = person_final_df.drop("personName")


#### Paso 4 - Escribir datos en Datalake en formato parquet

In [0]:
#overwrite_partition(person_final_df_dropped,"movie_silver","person","file_date")
merge_condition = 'tgt.person_id = src.person_id AND tgt.file_date = src.file_date'
merge_delta_lake(person_final_df_dropped,'movie_silver','person',silver_folder_path,merge_condition,'file_date')

In [0]:
%sql
SELECT file_date, COUNT(1) 
FROM movie_silver.person
GROUP BY file_date;